In [1]:
#montamos nuestro drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install python-docx langdetect stanza transformers

In [3]:
import docx
from langdetect import detect, DetectorFactory
from glob import glob
import os

In [4]:
import stanza
stanza.download('en')
# This class tokenize
nlp = stanza.Pipeline(lang='en', processors='tokenize')  

INFO:stanza:Downloading default packages for language: en (English)...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Done loading processors!


In [5]:
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
import torch
from pathlib import Path
from scipy.special import softmax
import numpy as np
import pandas as pd
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

In [12]:
file_path = r"https://www.dropbox.com/s/42ottszmsunrf18/sample_data.csv?dl=0"
df = pd.read_csv( file_path )

### Importing the transformer

In [ ]:
### asdasdsdasdas

In [ ]:
# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='offensive'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

### Auxiliar function

In [ ]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

### Example of Transformer operations

In [61]:
# Extracting text
text = "Good night 😊"
text = preprocess(text)

# Evaluating
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

# Getting Scores and Ranking
scores = output[0][0].detach().numpy()
scores = softmax(scores)
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) not-offensive 0.9073
2) offensive 0.0927


### Excuting transformer for all the posts

In [62]:
df.head()

,raw_post,female,title_id,post_id,topic,offensive,not-offensive
0,Really? I think women love sex.,1,102195,5,i-dont-think-that-women-really-like-sex,0.6666,0.3334
1,https://www.bbc.com/news/world-europe-28910662...,0,168828,2,european-gps-galileo-satellites-launched-into-...,0.1252,0.8748
2,The sociological commentary is strong here. Of...,0,25901,4,millie-bobby-brown-gets-best-actor-award,0.2970,0.7030
3,So they can throw them away once they get thei...,1,99821,2,why-do-guys-seduce-girls,0.2480,0.7520
4,"Look man, I'm not knocking restat at all. It's...",0,210244,15,this-question-crushed-me-at-my-flyout,0.6001,0.3999


In [ ]:
 for index, row in df.iterrows():
    
  # Post
  post = row[ "raw_post" ]
  
  doc = docx.Document()
  doc.add_paragraph( row['raw_post'] )
  fullText = []
  for para in doc.paragraphs:
    fullText.append(para.text)
  fullText = ' . '.join(fullText)
  # Get all the sentences in the doc file
  doc = nlp( fullText )

  # this list store all the sentences
  all_sentences_text = []
  for sentence in doc.sentences:
    
    # Get text
    sentence_text = ' '.join([token.text for token in sentence.tokens])
    
    # append sentence
    if not sentence_text.strip() == '' :
      all_sentences_text.append( sentence_text )
  
  #número máximo de palabras por sentencia: 512
  all_sentences_text = [ sentence for sentence in all_sentences_text if len( sentence ) < 512 ]                                                          

  # Defining text
  text = " ".join(all_sentences_text)

  try:
    # Evaluating
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)

    # Getting Scores and Ranking
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    for i in range(scores.shape[0]):
        l = labels[ranking[i]]
        s = scores[ranking[i]]
        # print(f"{i+1}) {l} {np.round(float(s), 4)}")

        df.loc[ index, l ] = np.round(float(s), 4)
    
  except:
    df.loc[ index, 'offensive' ] = np.nan
    df.loc[ index, 'not-offensive' ] = np.nan

Genereting means of the predictions

In [67]:
df.index

RangeIndex(start=0, stop=222405, step=1)

In [63]:
df.head()

,raw_post,female,title_id,post_id,topic,offensive,not-offensive
0,Really? I think women love sex.,1,102195,5,i-dont-think-that-women-really-like-sex,0.6666,0.3334
1,https://www.bbc.com/news/world-europe-28910662...,0,168828,2,european-gps-galileo-satellites-launched-into-...,0.1252,0.8748
2,The sociological commentary is strong here. Of...,0,25901,4,millie-bobby-brown-gets-best-actor-award,0.2970,0.7030
3,So they can throw them away once they get thei...,1,99821,2,why-do-guys-seduce-girls,0.2480,0.7520
4,"Look man, I'm not knocking restat at all. It's...",0,210244,15,this-question-crushed-me-at-my-flyout,0.6001,0.3999


In [ ]:
# New text

In [65]:
df1.shape

(5105, 7)

In [64]:
df1 = df.dropna().copy()
df1.groupby( ['female'], as_index = False )[['offensive',	'not-offensive']].mean()

,female,offensive,not-offensive
0,0,0.291625,0.708375
1,1,0.350956,0.649044
